In [1]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test set
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB

In [2]:
movie_train = pd.read_csv('data/imdb-train.csv')
movie_train = movie_train[['review','rating']]

In [3]:
movie_test = pd.read_csv('data/imdb-test.csv',error_bad_lines=False)
movie_test = movie_test[['review']]

b'Skipping line 12502: expected 1 fields, saw 2\nSkipping line 12503: expected 1 fields, saw 2\nSkipping line 12504: expected 1 fields, saw 2\nSkipping line 12505: expected 1 fields, saw 2\nSkipping line 12506: expected 1 fields, saw 2\nSkipping line 12507: expected 1 fields, saw 2\nSkipping line 12508: expected 1 fields, saw 2\nSkipping line 12509: expected 1 fields, saw 2\nSkipping line 12510: expected 1 fields, saw 2\nSkipping line 12511: expected 1 fields, saw 2\nSkipping line 12512: expected 1 fields, saw 2\nSkipping line 12513: expected 1 fields, saw 2\nSkipping line 12514: expected 1 fields, saw 2\nSkipping line 12515: expected 1 fields, saw 2\nSkipping line 12516: expected 1 fields, saw 2\nSkipping line 12517: expected 1 fields, saw 2\nSkipping line 12518: expected 1 fields, saw 2\nSkipping line 12519: expected 1 fields, saw 2\nSkipping line 12520: expected 1 fields, saw 2\nSkipping line 12521: expected 1 fields, saw 2\nSkipping line 12522: expected 1 fields, saw 2\nSkipping li

In [4]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [5]:
movie_train

review rating
0      Bromwell High is a cartoon comedy. It ran at t...    pos
1      Homelessness (or Houselessness as George Carli...    pos
2      Brilliant over-acting by Lesley Ann Warren. Be...    pos
3      This is easily the most underrated film inn th...    pos
4      This is not the typical Mel Brooks film. It wa...    pos
5      This isn't the comedic Robin Williams, nor is ...    pos
6      Yes its an art... to successfully make a slow ...    pos
7      In this "critically acclaimed psychological th...    pos
8      THE NIGHT LISTENER (2006) **1/2 Robin Williams...    pos
9      You know, Robin Williams, God bless him, is co...    pos
10     When I first read Armistead Maupins story I wa...    pos
11     I liked the film. Some of the action scenes we...    pos
12     There are many illnesses born in the mind of m...    pos
13     I enjoyed The Night Listener very much. It's o...    pos
14     The Night Listener is probably not one of Will...    pos
15     Like one of the previous commenters said, this...    pos
16     The Night Listener held my attention, with Rob...    pos
17     Popular radio storyteller Gabriel No one(Robin...    pos
18     If there is one thing to recommend about this ...    pos
19     I absolutely LOVED this film! I do not at all ...    pos
20     This Night Listener is better than people are ...    pos
21     When it comes to movies I can be pretty picky,...    pos
22     Somewhat funny and well-paced action thriller ...    pos
23     The legendary Boris Karloff ended his illustri...    pos
24     Liked Stanley & Iris very much. Acting was ver...    pos
25     I think the film is educational. However, it f...    pos
26     I don't know what movie some of these other pe...    pos
27     Despite reading the "initial comments" from so...    pos
28     As a convert into the Church of Jesus Christ o...    pos
29     This film is a true and historical film. It is...    pos
...                                                  ...    ...
24970  This film's premise is so simple and obvious t...    neg
24971  Considering how much money was budgeted for th...    neg
24972  Absolutely the worst film yet by Burton, who s...    neg
24973  We gave up at the point where George Clooney's...    neg
24974  Contrary to most other comments about "Syriana...    neg
24975  I saw this film last night following a lot of ...    neg
24976  OK Hollywood is not liberal.<br /><br />Obviou...    neg
24977  This movie, supposedly a thriller, had about f...    neg
24978  Exceedingly complicated and drab. I'm a bright...    neg
24979  I couldn't wait to receive the DVD after heari...    neg
24980  This movie was slower then Molasses in January...    neg
24981  Simply put: the movie is boring. Cliché upon c...    neg
24982  Alex D. Linz replaces Macaulay Culkin as the c...    neg
24983  The first Home Alone was a decent enough film....    neg
24984  'Home Alone 3' is the first of the Home Alone ...    neg
24985  Beating the bad guys... Again is the tag line ...    neg
24986  The first "Home Alone" was one of the funniest...    neg
24987  This movie was 100% boring, i swear i almost d...    neg
24988  This is the worst brain damaged, ultra cheap, ...    neg
24989  Some things need to be clarified. The picture ...    neg
24990  Invasion of the Star Creatures would definitel...    neg
24991  - Let me start by saying that I understand tha...    neg
24992  Gloria Victor and Dolores Reed in space girl c...    neg
24993  This was one of those films that got a ton of ...    neg
24994  I appreciate a think positive feel good about ...    neg
24995  Sondra Locke stinks in this film, but then she...    neg
24996  Clint Eastwood is Bronco Billy, the leader of ...    neg
24997  Sondra Locke stinks in this film, but then she...    neg
24998  This is a bit of a first for me, the first tim...    neg
24999  Ten minutes worth of story stretched out into ...    neg

[25000 rows x 2 columns]

In [6]:
import re
ps = PorterStemmer()
stops = set(stopwords.words("english"))

clean_train_reviews=[]

for review in movie_train["review"]:
    review = re.sub("[^a-zA-Z]"," ", review)
    words = review.lower().split()
    words = [w for w in words if not w in stops]
    words = [ps.stem(w) for w in words ]
    clean_train_reviews.append(" ".join(words))
  

In [7]:
clean_test_reviews=[]

for review in movie_test["review"]:
    review = re.sub("[^a-zA-Z]"," ", review)
    words = review.lower().split()
    words = [w for w in words if not w in stops]
    words = [ps.stem(w) for w in words ]
    clean_test_reviews.append(" ".join(words))
    

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vec = TfidfVectorizer(stop_words='english')
train_count=count_vec.fit_transform(clean_train_reviews)
test_count=count_vec.transform(clean_test_reviews)

In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
from sklearn.cross_validation import KFold,cross_val_score
scores=cross_val_score(clf,train_count,movie_train["rating"],cv=10,scoring='accuracy')

/Users/sachinmittal/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
scores.mean()

0.8013999999999999

In [11]:
clf1 = MultinomialNB().fit(train_count,movie_train["rating"])

In [12]:
# Predicting the Test set results, find accuracy
y_pred = clf1.predict(test_count)
y_pred

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'pos'], dtype='<U3')

In [13]:

y_pred = pd.DataFrame(y_pred,columns=['rating'])
frames=[movie_test,y_pred]
result=pd.concat(frames,axis=1)
result.to_csv("imdb-test.csv", sep='\t')